Recover influence links

1. Merge two influence lists: merge_influ_list(entInfo_list)
2. Map influence to influencedBy
3. Leverage other links in the pages
4. Influence across fields

Note:

https://stackoverflow.com/questions/46122397/querying-dbpedia-to-get-influenced-and-influence-by-for-a-particular-artist

http://mappings.dbpedia.org/index.php/Mapping_Guide#Check_for_similar_mappings


dbo:influenced = dbp:influenced - is dbp:influences of

dbo:influencedBy = dbp:influences - is dbp:influenced of

1. Merge

In [ ]:
from function import merge_db_influ_list

# merge the lists
ent_info_list = db_philo_info_list
db_philo_info_merged_list = merge_db_influ_list(ent_info_list)
pickle.dump( db_philo_info_merged_list, open( "info_lists/db_philo_info_merged_list.p", "wb" ) )

[] ['Pythagoras']
['Pythagoras']
[] ['Aristippus']
['Aristippus']
[] ['Zeno_of_Citium']
['Zeno_of_Citium']
[] ['Stilpo']
['Stilpo']
['Socrates', 'Plato'] ['Socrates']
['Plato', 'Socrates']
['Damascius', 'Synesius'] []
['Synesius', 'Damascius']
['Aristotle', 'Plato', 'Plotinus', 'Theon_of_Alexandria'] ['Ammonius_Saccas', 'Plotinus']
['Plato', 'Aristotle', 'Ammonius_Saccas', 'Theon_of_Alexandria', 'Plotinus']
['Simone_Weil', 'Petrarch', 'Peter_Abelard', 'Thomas_Aquinas'] ['Sandrine_Bergès']
['Thomas_Aquinas', 'Simone_Weil', 'Peter_Abelard', 'Petrarch', 'Sandrine_Bergès']
['Benedict_of_Nursia', 'Cicero', 'Jerome', 'Lucan', 'Plutarch', 'Tertullian'] []
['Tertullian', 'Cicero', 'Benedict_of_Nursia', 'Jerome', 'Plutarch', 'Lucan']
['Christian_mysticism', 'Brethren_of_the_Free_Spirit'] []
['Brethren_of_the_Free_Spirit', 'Christian_mysticism']
['William_of_St-Thierry', 'Augustine_of_Hippo', 'Bernard_of_Clairvaux', 'Richard_of_Saint_Victor', 'Boethius', 'Jean_de_Meun', 'Meister_Eckhart'] []
['W

2. Map

In [21]:
from function import invert_dict
from function import merge_dict
from function import map_db_influ_list

import pickle


ent_info_list = pickle.load( open("info_lists/db_philo_info_merged_list.p", "rb") )
influ_dict, influBy_dict = map_db_influ_list(ent_info_list)

pickle.dump( influ_dict, open( "info_lists/influ_dict.p", "wb" ) )
pickle.dump( influBy_dict, open( "info_lists/influBy_dict.p", "wb" ) )

influence merged stats:
dict: 1971 | inverse dict: 3615
influendBy merged stats:
dict: 1971 | inverse dict: 1493


In [22]:
# inclusion: philosophers or not?
ent_list = [i['entity'].replace(" ","_") for i in ent_info_list]
all_ent_list = list( set( list(influ_dict.keys()) + list(influBy_dict.keys()) ) )

non_philo_ent_list = [i for i in all_ent_list if i not in ent_list]
philo_ent_list = [i for i in all_ent_list if i in ent_list]

philo_influ_ent_list = [i for i in list(influ_dict.keys()) if i in philo_ent_list and influ_dict[i] != []]
philo_influBy_ent_list = [i for i in list(influBy_dict.keys()) if i in philo_ent_list and influBy_dict[i] != []]

print("non_philo_ent_list:",len(non_philo_ent_list), 
    "| philo_ent_list:",len(philo_ent_list),
    "\nphilo_influ_ent_list:",len(philo_influ_ent_list),
    "| philo_influBy_ent_list:",len(philo_influBy_ent_list))
print( len(all_ent_list) == len(non_philo_ent_list) + len(philo_ent_list) )

non_philo_ent_list: 3354 | philo_ent_list: 1971 
philo_influ_ent_list: 969 | philo_influBy_ent_list: 994
True


In [23]:
# update the mapped influence: db_philo_info_influ_conmpl_list
import copy
ent_info_list = pickle.load( open("info_lists/db_philo_info_merged_list.p", "rb") )
info_list = copy.deepcopy(ent_info_list)

for item in info_list:
    item['inferred_info'] = {
        'influence':[],
        'influencedBy':[]
    }

    if item["entity"] == "":
        continue
    ent = item["entity"].replace(" ","_")
    item['inferred_info']['influence'] = influ_dict[ent]
    item['inferred_info']['influencedBy'] = influBy_dict[ent]

# check
COUNT_1, COUNT_2 = 0, 0
a = []
for item in info_list:
    if item["entity"] != "" and item['inferred_info']['influence']: # has influence
        COUNT_1 += 1
        a.append(item['entity'])
    if item['inferred_info']['influencedBy']: # has influenced by
        COUNT_2 += 1
    
if COUNT_1 == len(philo_influ_ent_list) and COUNT_2 == len(philo_influBy_ent_list):
    print("Success: map_db_influ_list")
    print("influence:",COUNT_1, "| influencedBy:",COUNT_2)
    pickle.dump( info_list, open( "info_lists/db_philo_info_influ_conmpl_list.p", "wb" ) )
    print("Saved as info_lists/db_philo_info_influ_conmpl_list.p")
else:
    print("influence:",COUNT_1, "| influencedBy:",COUNT_2)

Success: map_db_influ_list
influence: 969 | influencedBy: 994
Saved as info_lists/db_philo_info_influ_conmpl_list.p


Add ID and other info to influence lists

In [24]:
# make influ links as dictionaries, with id and name
import function
from function import add_info_influ_dict

ent_info_list = pickle.load( open("info_lists/db_philo_info_influ_conmpl_list.p", "rb") )

# ent_info_list[0].keys()
# for item in ent_info_list:
#         if item['inferred_info']['influence']:
#             print(item['entity'])

db_philo_info_infludict_list = add_info_influ_dict(ent_info_list)
pickle.dump( db_philo_info_infludict_list, open( "info_lists/db_philo_info_infludict_list.p", "wb" ) )


Influence updated
InfluencedBy updated


## Find philosophers

## Summary

In [76]:
# open "info_lists/db_philo_info_influ_conmpl_list.p"
with open('info_lists/db_philo_info_influ_conmpl_list.p', 'rb') as fp:
    db_philo_info_influ_conmpl_list = pickle.load(fp)

ent_list = [i['entity'].replace(" ","_") for i in db_philo_info_influ_conmpl_list]

# count the number of influence and influencedBy
for item in db_philo_info_influ_conmpl_list:

    item["influ_count"] = {}
    count_influence, count_influencedBy, count_influence_inferred, count_influencedBy_inferred = 0, 0, 0, 0
    # Decision: I only take those considered as philosophers (have id)

    count_influence = len([i for i in item['dbpedia_info']['db_influence_record']['influences'] if i in ent_list])
    count_influencedBy = len([i for i in item['dbpedia_info']['db_influence_record']['influencedBy'] if i in ent_list])
    count_influence_inferred = len([i for i in item['inferred_info']['influence'] if i in ent_list])
    count_influencedBy_inferred = len([i for i in item['inferred_info']['influencedBy'] if i in ent_list])

    # count_influence = len(item['dbpedia_info']['db_influence_record']['influences'])
    # count_influencedBy = len(item['dbpedia_info']['db_influence_record']['influencedBy'])
    # count_influence_inferred = len(item['inferred_info']['influence'])
    # count_influencedBy_inferred = len(item['inferred_info']['influencedBy'])
    item["influ_count"] = {
        'influence':count_influence,
        'influencedBy':count_influencedBy,
        'influence_inferred':count_influence_inferred,
        'influencedBy_inferred':count_influencedBy_inferred,
        "influ_update":  count_influence_inferred - count_influence,
        "influBy_update": count_influencedBy_inferred - count_influencedBy
    }
    # sort
    item['dbpedia_info']['db_influence_record']['influencedBy'] = sorted(item['dbpedia_info']['db_influence_record']['influencedBy'])
    item['dbpedia_info']['db_influence_record']['influences'] = sorted(item['dbpedia_info']['db_influence_record']['influences'])
    item['inferred_info']['influence'] = sorted(item['inferred_info']['influence'])
    item['inferred_info']['influencedBy'] = sorted(item['inferred_info']['influencedBy'])

    print(item['entity'], count_influence, count_influencedBy, count_influence_inferred, count_influencedBy_inferred)

# save
pickle.dump( db_philo_info_influ_conmpl_list, open( "info_lists/db_philo_info_count_list.p", "wb" ) )


Lopamudra 0 0 0 0
Maitreyi 0 0 0 0
Ghosha 0 0 0 0
Gargi Vachaknavi 0 0 0 0
Theano of Croton 0 0 0 0
Aristoclea of Delphi 0 0 1 0
Khujjuttarā 0 0 0 0
Aspasia of Miletus 0 0 0 0
Arete of Cyrene 0 0 0 1
Hipparchia of Maroneia 0 1 1 1
Nicarete of Megara 0 0 0 1
Catherine of Alexandria 0 0 0 0
Ptolemais of Cyrene 0 0 0 0
Aesara of Lucania 0 0 0 0
Diotima of Mantinea 2 0 2 0
Ban Zhao 0 0 0 0
Sosipatra of Ephesus 0 0 0 0
Xie Daoyun 0 0 0 0
Hypatia 1 3 1 4
Aedesia 0 0 0 0
Theodora 0 0 0 0
Ubhaya Bharati 0 0 0 0
Héloïse d'Argenteuil 4 3 4 3
Akka Mahadevi 0 0 0 0
Marguerite Porete 0 3 0 3
Tullia d'Aragona 0 0 0 0
Lalleshwari 0 0 0 0
Catherine of Siena 0 0 0 0
Moderata Fonte 0 0 0 0
Hildegard of Bingen 0 0 0 0
Teresa of Ávila 0 0 3 0
Mary Astell 0 0 0 0
Damaris Cudworth Masham 4 6 5 6
Laura Bassi 0 1 0 1
Harriet Martineau 0 0 0 1
Helena Blavatsky 1 0 3 0
Frances Power Cobbe 0 0 0 0
Antoinette Brown Blackwell 0 0 0 0
Margaret Cavendish 0 0 0 0
Émilie du Châtelet 0 1 0 3
Sister Nivedita 0 0 0 0
Cat

In [82]:
# open "info_lists/db_philo_info_count_list.p"
with open('info_lists/db_philo_info_count_list.p', 'rb') as fp:
    db_philo_info_count_list = pickle.load(fp)


# initialize the list of philosophers with updated influence and influenced by
influ_update_philo_list = []
influBy_update_philo_list = []
both_update_philo_list = []

# Assumption: if the lengths are the same, then the lists are the same

# print the first item whose ['inferred_info']['influence'] is not equal to ['dbpedia_info']['db_influence_record']['influence_merged']
for item in db_philo_info_count_list:
    if item['influ_count']['influ_update'] != 0 and item['influ_count']['influBy_update'] != 0:
        both_update_philo_list.append(item)
    # if item['inferred_info']['influence'] != item['dbpedia_info']['db_influence_record']['influences']:
    else:
        if item['influ_count']['influ_update'] != 0:
            influ_update_philo_list.append(item)

        # if item['inferred_info']['influencedBy'] != item['dbpedia_info']['db_influence_record']['influencedBy']:
        elif item['influ_count']['influBy_update'] != 0:
            influBy_update_philo_list.append(item)

    
print("influ_update_philo_list:",len(influ_update_philo_list))
print("influBy_update_philo_list:",len(influBy_update_philo_list))
print("both_update_philo_list:",len(both_update_philo_list))

influ_update_philo_list = sorted(influ_update_philo_list, key=lambda k: k['influ_count']['influ_update'], reverse=True)
influBy_update_philo_list = sorted(influBy_update_philo_list, key=lambda k: k['influ_count']['influBy_update'], reverse=True)
both_update_philo_list = sorted(both_update_philo_list, key=lambda k: k['influ_count']['influ_update'], reverse=True)

count_women_influ, count_women_influBy, count_women_both, count_women_isolated = 0, 0, 0, 0

print("influ =================================================================")

for item in influ_update_philo_list:
    if "women: " in item["inclusion"]:
        print("&&&&&&&&&&& Women philosopher ")
        count_women_influ += 1
    print(list(item["influ_count"].values())[-2:], item["entity"], list(item["influ_count"].values())[:4])
        
print("influBy =================================================================")

for item in influBy_update_philo_list:
    if "women: " in item["inclusion"]:
        print("&&&&&&&&&&& Women philosopher ")
        count_women_influBy += 1
    print(list(item["influ_count"].values())[-2:], item["entity"], list(item["influ_count"].values())[:4])

print("influ & influBy =================================================================")
for item in both_update_philo_list:
    if "women: " in item["inclusion"]:
        print("&&&&&&&&&&& Women philosopher ")
        count_women_both += 1
    print(list(item["influ_count"].values())[-2:], item["entity"], list(item["influ_count"].values())[:4])

# for people in all these list, print and count those who previously have no influence and influencedBy
count_isolated = 0
for item in influ_update_philo_list:
    if item['influ_count']['influence'] == 0 and item['influ_count']['influencedBy'] == 0:
        if "women: " in item["inclusion"]:
            print("&&&&&&&&&&& Women philosopher", list(item["influ_count"].values())[-2:], item["entity"], list(item["influ_count"].values())[:4])
            count_women_isolated += 1
        else:
            print(list(item["influ_count"].values())[-2:], item["entity"], list(item["influ_count"].values())[:4])
        count_isolated += 1
print("count_isolated:", count_isolated)

# for all women philosophers, print the percentage of those who previously have no influence and influencedBy
print(count_women_influ, count_women_influBy, count_women_both, count_women_isolated)

print("%%%%%%%%%%%%%%%%%%%%%%%% women philosophers whose influences and influencers are not updated")
count_women_noupdate, count_women_remain_isolated = 0, 0
# count women philosophers who have no influence and influencedBy
for item in db_philo_info_count_list:
    if "women: " in item["inclusion"] and item['influ_count']['influ_update'] == 0 and item['influ_count']['influBy_update'] == 0:
        count_women_noupdate += 1
        if item['influ_count']['influence'] == 0 and item['influ_count']['influencedBy'] == 0:
            count_women_remain_isolated += 1
        print(list(item["influ_count"].values())[-2:], item["entity"], list(item["influ_count"].values())[:4])
print(count_women_noupdate, count_women_remain_isolated)

influ_update_philo_list: 211
influBy_update_philo_list: 298
both_update_philo_list: 329
influ =================================================================
[11, 0] Blaise Pascal [5, 5, 16, 5]
[8, 0] Al-Farabi [8, 4, 16, 4]
[8, 0] Homer [0, 0, 8, 0]
[7, 0] Hermann Cohen [4, 2, 11, 2]
[7, 0] Heraclitus of Ephesus [13, 2, 20, 2]
[6, 0] Fyodor Dostoyevsky [0, 0, 6, 0]
[5, 0] John Langshaw Austin [4, 6, 9, 6]
[5, 0] Jakob Böhme [4, 1, 9, 1]
[5, 0] Confucius [0, 0, 5, 0]
[5, 0] Hugo Grotius [7, 3, 12, 3]
[5, 0] Johann Georg Hamann [6, 5, 11, 5]
[5, 0] Friedrich Hölderlin [0, 0, 5, 0]
[5, 0] Niccolò Machiavelli [16, 8, 21, 8]
[4, 0] Siddhartha Gautama [0, 0, 4, 0]
[4, 0] Johann Friedrich Herbart [5, 2, 9, 2]
[4, 0] William Ernest Johnson [0, 0, 4, 0]
[4, 0] Clarence Irving Lewis [0, 3, 4, 3]
[4, 0] Rosa Luxemburg [0, 0, 4, 0]
[4, 0] Ralph Barton Perry [0, 0, 4, 0]
[4, 0] Jean Piaget [1, 4, 5, 4]
[4, 0] George Santayana [0, 7, 4, 7]
[4, 0] Emanuel Swedenborg [0, 0, 4, 0]
[4, 0] Thucydides 

In [123]:
# for all items, summarize every variable in ['influ_count']

import plotly
# use plot to print overlay histogram
import plotly.figure_factory as ff
import plotly.graph_objs as go
import pandas as pd
# use plotly to create graphs that lay beside each other
from plotly.subplots import make_subplots



# for 'influence'
#         'influencedBy'
#         'influence_inferred'
#         'influencedBy_inferred'
#         "influ_update"
#         "influBy_update"
#     }

def describe_count(key_list, db_philo_info_count_list):
    # use plotly to print an overlay histogram of all keys in key_list
    # overlay histogram using go
    fig = go.Figure()

    # use plotly to create graphs that lay beside each other
    # rows are three status
    # cols are two variables
    fig = make_subplots(rows=3, cols=2, subplot_titles=(
        "\"Influence\" Links Before Update", "\"Influenced By\" Links Before Update", 
        "\"Influence\" Links After Update", "\"Influenced By\" Links After Update", 
        "\"Influence\" Links Recovered", "\"Influenced By\" Links Recovered"
        ))
    # put the six graphs in the same figure, put in each row and column
    for i in range(3):
        for j in range(2):
            # print(i, j)
            # count zero values for each key
            count_zero = 0
            for item in db_philo_info_count_list:
                if item['influ_count'][key_list[i*2+j]] == 0:
                    count_zero += 1
            print(key_list[i*2+j], "count_zero:", count_zero)
            # ignore zero values and create a new df
            df = pd.DataFrame([item['influ_count'][key_list[i*2+j]] for item in db_philo_info_count_list if item['influ_count'][key_list[i*2+j]] != 0], columns=[key_list[i*2+j]])
            # use the df to create a histogram
            fig.add_trace(go.Histogram(x=df[key_list[i*2+j]]), row=i+1, col=j+1)

            # fig.add_trace(go.Histogram(x=[item['influ_count'][key_list[i*2+j]] for item in db_philo_info_count_list]), row=i+1, col=j+1)
    fig.update_layout(barmode='overlay')
    # change the font
    fig.update_layout(font=dict(family="calibri", size=18, color="#7f7f7f"))
    # decrease the distance between each subplot
      
    # decrease the font size of xaxis and yaxis
    fig.update_xaxes(title_font=dict(size=12), tickfont=dict(size=12))
    fig.update_yaxes(title_font=dict(size=12), tickfont=dict(size=12))
    # change bin width
    fig.update_layout(bargap=0.05)
    fig.update_traces(opacity=0.75)
    fig.show()
    # save as html
    plotly.offline.plot(fig, filename='vis/histogram_for_links.html')

# a function that find all philosophers who have no influence and influencedBy
def find_isolated(db_philo_info_count_list, status):
    if status == "before_update":
        key_list = ['influence', 'influencedBy']
    elif status == "after_update":
        key_list = ['influence_inferred', 'influencedBy_inferred']
    elif status == "update":
        key_list = ['influ_update', 'influBy_update']
    else:
        print("wrong status")
        return
    
    count_isolated = 0
    for item in db_philo_info_count_list:
        if item['influ_count'][key_list[0]] == 0 and item['influ_count'][key_list[1]] == 0:
            # print(list(item["influ_count"].values())[-2:], item["entity"], list(item["influ_count"].values())[:4])
            count_isolated += 1
    print(status, "count_isolated:", count_isolated)


find_isolated(db_philo_info_count_list, "before_update")
find_isolated(db_philo_info_count_list, "after_update")
find_isolated(db_philo_info_count_list, "update")


key_list = ['influence', 'influencedBy', 'influence_inferred', 'influencedBy_inferred', 'influ_update', 'influBy_update']

describe_count(key_list, db_philo_info_count_list)



before_update count_isolated: 1365
after_update count_isolated: 958
update count_isolated: 1173
influence count_zero: 1599
influencedBy count_zero: 1398
influence_inferred count_zero: 1286
influencedBy_inferred count_zero: 1100
influ_update count_zero: 1471
influBy_update count_zero: 1384


For women philosophers
before_update count_isolated: 189
after_update count_isolated: 163
update count_isolated: 201
influence count_zero: 238
influencedBy count_zero: 192
influence_inferred count_zero: 224
influencedBy_inferred count_zero: 172
influ_update count_zero: 240
influBy_update count_zero: 213


In [127]:
# count all the updated influence and influenced by links
count_influ_update = 0
count_influBy_update = 0
for item in db_philo_info_count_list:
    count_influ_update += item['influ_count']['influ_update']
    count_influBy_update += item['influ_count']['influBy_update']
print("count_influ_update:", count_influ_update)
print("count_influBy_update:", count_influBy_update)

# count all influence and influenced by links
count_influence = 0
count_influencedBy = 0
for item in db_philo_info_count_list:
    count_influence += item['influ_count']['influence']
    count_influencedBy += item['influ_count']['influencedBy']
print("count_influence:", count_influence)
print("count_influencedBy:", count_influencedBy)

# count all inferred influence and influenced by links
count_influence_inferred = 0
count_influencedBy_inferred = 0
for item in db_philo_info_count_list:
    count_influence_inferred += item['influ_count']['influence_inferred']
    count_influencedBy_inferred += item['influ_count']['influencedBy_inferred']
print("count_influence_inferred:", count_influence_inferred)
print("count_influencedBy_inferred:", count_influencedBy_inferred)

count_influ_update: 2259
count_influBy_update: 1455
count_influence: 1776
count_influencedBy: 2580
count_influence_inferred: 4035
count_influencedBy_inferred: 4035


Number of nodes 2012
Total number of Total influence links 1776 =2259=> 4035
Total influenced by links 2580 =1455=> 4035
 
Philosophers with only “influence” links updated 211
Philosophers with only “influenced by” links updated 298
Philosophers with both links updated 329

Philosophers with no influence links 1599 =310=> 1289
Philosophers with no influenced by links 1398 =298=> 1100
Isolated philosophers before correcting links 1365 =407=> 958

====== Women

Number of nodes 261
Total number of influence links 56=40=> 96
Total influenced by links 232=75=> 307

Philosophers with no influence links 238=14=>224
Philosophers with no influenced by links 192=20=>172
Isolated philosophers before correcting links 189=26=> 163


In [128]:
# women philosophers
print("%%%%%%%%%%%%%%%% For women philosophers %%%%%%%%%%%%%%%%%%%")
women_info_count_list = [item for item in db_philo_info_count_list if "women" in item["inclusion"]]
find_isolated(women_info_count_list, "before_update")
find_isolated(women_info_count_list, "after_update")
find_isolated(women_info_count_list, "update")
describe_count(key_list, women_info_count_list)

# count all the updated influence and influenced by links
count_influ_update = 0
count_influBy_update = 0
for item in women_info_count_list:
    count_influ_update += item['influ_count']['influ_update']
    count_influBy_update += item['influ_count']['influBy_update']
print("count_influ_update:", count_influ_update)
print("count_influBy_update:", count_influBy_update)

# count all influence and influenced by links
count_influence = 0
count_influencedBy = 0
for item in women_info_count_list:
    count_influence += item['influ_count']['influence']
    count_influencedBy += item['influ_count']['influencedBy']
print("count_influence:", count_influence)
print("count_influencedBy:", count_influencedBy)

# count all inferred influence and influenced by links
count_influence_inferred = 0
count_influencedBy_inferred = 0
for item in women_info_count_list:
    count_influence_inferred += item['influ_count']['influence_inferred']
    count_influencedBy_inferred += item['influ_count']['influencedBy_inferred']
print("count_influence_inferred:", count_influence_inferred)
print("count_influencedBy_inferred:", count_influencedBy_inferred)

%%%%%%%%%%%%%%%% For women philosophers %%%%%%%%%%%%%%%%%%%
before_update count_isolated: 189
after_update count_isolated: 163
update count_isolated: 201
influence count_zero: 238
influencedBy count_zero: 192
influence_inferred count_zero: 224
influencedBy_inferred count_zero: 172
influ_update count_zero: 240
influBy_update count_zero: 213


count_influ_update: 40
count_influBy_update: 75
count_influence: 56
count_influencedBy: 232
count_influence_inferred: 96
count_influencedBy_inferred: 307


In [130]:
# count all nodes that can be linked to women philsoophers' nodes

# identify women philosophers' influence

# item['dbpedia_info']['db_influence_record']['influencedBy'] = sorted(item['dbpedia_info']['db_influence_record']['influencedBy'])
#     item['dbpedia_info']['db_influence_record']['influences'] = sorted(item['dbpedia_info']['db_influence_record']['influences'])
#     item['inferred_info']['influence'] = sorted(item['inferred_info']['influence'])
#     item['inferred_info']['influencedBy'] = sorted(item['inferred_info']['influencedBy'])


influence_list, influenced_by_list = [], []
influence_update_list, influenced_by_update_list = [], []
for item in women_info_count_list:
    influence_list += item['dbpedia_info']['db_influence_record']['influences']
    influenced_by_list += item['dbpedia_info']['db_influence_record']['influencedBy']
    influence_update_list += item['inferred_info']['influence']
    influenced_by_update_list += item['inferred_info']['influencedBy']
influence_list = sorted(list(set(influence_list)))
influenced_by_list = sorted(list(set(influenced_by_list)))

all_links = influence_list + influenced_by_list
all_links = sorted(list(set(all_links)))
print("all_links:", len(all_links))
all_links_updated = influence_update_list + influenced_by_update_list
all_links_updated = sorted(list(set(all_links_updated)))
print("all_links_updated:", len(all_links_updated))

influence_update_list = sorted(list(set(influence_update_list)))
influenced_by_update_list = sorted(list(set(influenced_by_update_list)))
print("influence_list:", len(influence_list),
      "influenced_by_list:", len(influenced_by_list),
        "influence_update_list:", len(influence_update_list),
        "influenced_by_update_list:", len(influenced_by_update_list))
        


all_links: 273
all_links_updated: 414
influence_list: 112 influenced_by_list: 187 influence_update_list: 207 influenced_by_update_list: 245


In [69]:
# from db_philo_info_count_list
# "Ronald Dworkin"
for item in db_philo_info_count_list:
    if item['entity'] == 'Gottfried Leibniz':
        # print(item)
        # print(item['inferred_info']['influence'])
        print(item['dbpedia_info']['db_influence_record']['influences'])
        # # print the difference between the two list
        print(set(item['inferred_info']['influence']) - set(item['dbpedia_info']['db_influence_record']['influences']))

        print()
        # print(item['inferred_info']['influencedBy'])
        print(item['dbpedia_info']['db_influence_record']['influencedBy'])
        print(set(item['inferred_info']['influencedBy']) - set(item['dbpedia_info']['db_influence_record']['influencedBy']))



['Alfred_Tarski', 'Benoit_Mandelbrot', 'Bernhard_Riemann', 'Bertrand_Russell', 'Carl_Friedrich_Gauss', 'Charles_Sanders_Peirce', 'Christian_Wolff_(philosopher)', 'David_Hume', 'Edward_Gibbon', 'Ernst_Platner', 'George_Berkeley', 'George_Boole', 'Gilles_Deleuze', 'Gottlob_Frege', 'Immanuel_Kant', 'Jean-Jacques_Rousseau', 'John_Henry_Newman', 'Joseph-Louis_Lagrange', 'Kurt_Gödel', 'Leonhard_Euler', 'Martin_Heidegger', 'Maurice_Blondel', 'Nicholas_Rescher', 'Norbert_Wiener', 'Voltaire', 'Walter_Rauschenbusch', 'Wilhelm_Wundt']
{'Mookie_Tenembaum', 'Maine_de_Biran', 'Emer_de_Vattel', 'Xavier_Zubiri', 'J._L._Austin', 'Robert_Merrihew_Adams', 'Harald_Høffding', 'Mário_Ferreira_dos_Santos', 'Ernst_Cassirer', 'Gustav_Teichmüller', 'M._M._Sharif', 'Henri_Bergson', 'Branislav_Petronijević', 'Alexander_Pruss', 'Friedrich_Wilhelm_Joseph_Schelling', 'Georges_Friedmann', 'Friedrich_Heinrich_Jacobi', 'Lorenzo_Peña', 'Louis_de_Bonald', 'Gilles_Châtelet', 'Dan_Goldstick', 'Pierre_Louis_Maupertuis', 'Fé

## 

## Get some statistics

In [29]:
import pickle

import function
from function import print_recordlink_stats
from function import print_inferlink_stats


# before mapping
# after mapped
ent_info_list = pickle.load( open("info_lists/db_philo_info_influ_conmpl_list.p", "rb") )
print_recordlink_stats(ent_info_list)
print_inferlink_stats(ent_info_list)


influ_dict = pickle.load( open("info_lists/influ_dict.p", "rb") )
influBy_dict = pickle.load( open("info_lists/influBy_dict.p", "rb") )


print("people who influenced others", len(influ_dict))
print("people who were influenced by others", len(influBy_dict))

people who influenced others 952
people who were influenced by others 952
In total, influence links 9220 | influencedBy links 4884


people who influenced others 969
people who were influenced by others 994
In total, influence links 9908 | influencedBy links 5640


people who influenced others 2857
people who were influenced by others 4796


In [1]:
import plotly.express as px
# import plotly.graph_objects as go

import pickle
import numpy as np
import pandas as pd


In [ ]:
# Summarize influence and influencedby links